# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.381584     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.514618     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
feature_weights = model_all.coefficients

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

In [7]:
non_zero_features = feature_weights[feature_weights['value'] != 0]

print(non_zero_features[1:]['name'])

['bathrooms', 'sqft_living', 'sqft_living_sqrt', 'grade', 'sqft_above']


# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [9]:
import numpy as np

l1_penalty_values = np.logspace(1, 7, num=13)
RSS = []
for l1_penalty in np.logspace(1, 7, num=13):
    model = turicreate.linear_regression.create(training, target='price', features=all_features, validation_set=None,
                                               l2_penalty=0, l1_penalty=l1_penalty, verbose=False)
    predictions = model.predict(validation)
    residuals = predictions - validation['price']
    rss = (residuals * residuals).sum()
    RSS.append(rss)

In [34]:
print(turicreate.SFrame({'L1 Penalty Values':l1_penalty_values, 'RSS':RSS}))

+--------------------+-------------------+
| L1 Penalty Values  |        RSS        |
+--------------------+-------------------+
|        10.0        | 625766285142459.9 |
| 31.622776601683793 | 625766285362394.1 |
|       100.0        | 625766286057885.0 |
| 316.22776601683796 | 625766288257224.8 |
|       1000.0       | 625766295212186.6 |
| 3162.2776601683795 | 625766317206080.5 |
|      10000.0       | 625766386760658.1 |
| 31622.776601683792 | 625766606749278.5 |
|      100000.0      | 625767302791634.1 |
| 316227.7660168379  | 625769507643886.2 |
+--------------------+-------------------+
[13 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [41]:
index = np.argmin(RSS)
print('The best value for the l1_penalty = {}'.format(l1_penalty_values[index]))

The best value for the l1_penalty = 10.0


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [11]:
model_best_l1_penalty = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                           l2_penalty=0, l1_penalty=10.0, validation_set=None,
                                                           verbose=False)
print(model_best_l1_penalty.coefficients)

+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|   (intercept)    |  None |  18993.427212770577  |  None  |
|     bedrooms     |  None |  7936.967679031306   |  None  |
| bedrooms_square  |  None |   936.993368193299   |  None  |
|    bathrooms     |  None |  25409.58893412067   |  None  |
|   sqft_living    |  None |  39.11513637970762   |  None  |
| sqft_living_sqrt |  None |  1124.6502128077207  |  None  |
|     sqft_lot     |  None | 0.003483618222989743 |  None  |
|  sqft_lot_sqrt   |  None |  148.2583910114082   |  None  |
|      floors      |  None |  21204.33546695013   |  None  |
|  floors_square   |  None |  12915.524336072436  |  None  |
+------------------+-------+----------------------+--------+
[18 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and colum

In [12]:
print('Number of nonzero weights = {}'.format(model_best_l1_penalty.coefficients['value'].nnz()))

Number of nonzero weights = 18


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [13]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [14]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [31]:
nnz = []
for l1_penalty in np.logspace(8, 10, num=20):
    model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                               l1_penalty=l1_penalty, l2_penalty=0, validation_set=None, verbose=False)
    nnz_values = model.coefficients['value'].nnz()
    nnz.append(nnz_values)
print(nnz)

[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [32]:
l1_penalty_values = list(l1_penalty_values)
l1_penalty_nnz = dict(zip(l1_penalty_values, nnz))

l1_penalty_min_dict = dict()
for (key, value) in l1_penalty_nnz.items():
    if value > max_nonzeros:
        l1_penalty_min_dict[key] = value

l1_penalty_max_dict = dict()
for (key, value) in l1_penalty_nnz.items():
    if value < max_nonzeros:
        l1_penalty_max_dict[key] = value
        

In [33]:
l1_penalty_min = max(l1_penalty_min_dict.keys())
l1_penalty_max = min(l1_penalty_max_dict.keys())

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

In [34]:
print('l1_penalty_min = ' + str(l1_penalty_min))
print('l1_penalty_max = ' + str(l1_penalty_max))

l1_penalty_min = 2976351441.6313133
l1_penalty_max = 3792690190.7322536


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [35]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [37]:
RSS = []
nnz = []

for l1_penalty in np.linspace(l1_penalty_min, l1_penalty_max, num=20):
    model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                               l1_penalty=l1_penalty, l2_penalty=0, validation_set=None, verbose=False)
    predictions = model.predict(validation)
    residuals = predictions - validation['price']
    rss = (predictions * predictions).sum()
    RSS.append(rss)
    nnz_values = model.coefficients['value'].nnz()
    nnz.append(nnz_values)

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [46]:
l1_penalty_values_RSS = dict(zip(l1_penalty_values, RSS))
l1_penalty_values_nnz = dict(zip(l1_penalty_values, nnz))

l1_penalty_max_nonzeros = dict()
for (key, value) in l1_penalty_values_nnz.items():
    if value == max_nonzeros:
        l1_penalty_max_nonzeros[key] = value
        
l1_penalty_max_nonzeros_RSS = dict()
for (key, value) in l1_penalty_values_RSS.items():
    for(k, v) in l1_penalty_max_nonzeros.items():
        if key == k:
            l1_penalty_max_nonzeros_RSS[key] = value
            
best_l1_penalty = max(l1_penalty_max_nonzeros_RSS, key=l1_penalty_max_nonzeros_RSS.get)
print('The l1_penalty in our narrow range that has the lowest RSS on the VALIDATION set\nand has sparsity equal to max_nonzeros = ' + str(best_l1_penalty))

The l1_penalty in our narrow range that has the lowest RSS on the VALIDATION set
and has sparsity equal to max_nonzeros = 3448968612.163437


In [48]:
model_best_l1_penalty = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                           l1_penalty=best_l1_penalty, l2_penalty=0,
                                                           validation_set=None, verbose=False)
print(model_best_l1_penalty.coefficients)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 222253.19254432785 |  None  |
|     bedrooms     |  None | 661.7227177822587  |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None | 15873.957259267981 |  None  |
|   sqft_living    |  None | 32.41022145125964  |  None  |
| sqft_living_sqrt |  None | 690.1147733133256  |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None |        0.0         |  None  |
|  floors_square   |  None |        0.0         |  None  |
+------------------+-------+--------------------+--------+
[18 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [49]:
feature_weights = model_best_l1_penalty.coefficients
non_zero_features = feature_weights[feature_weights['value'] != 0]

print(non_zero_features[1:]['name'])

['bedrooms', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'grade', 'sqft_above']
